##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrating tf.summary usage to TF 2.x

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/migrate"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/migrate.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/migrate.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/migrate.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

> Note: This doc is for people who are already familiar with TensorFlow 1.x TensorBoard and who want to migrate large TensorFlow code bases from TensorFlow 1.x to 2.x. If you're new to TensorBoard, see the [get started](get_started.ipynb) doc instead. If you are using `tf.keras` there may be no action you need to take to upgrade to TensorFlow 2.x.


In [ ]:
import tensorflow as tf

TensorFlow 2.x includes significant changes to the `tf.summary` API used to write summary data for visualization in TensorBoard.

## 変更点

`tf.summary` API を 2 つのサブ API として考えると良いでしょう。

- 個別の要約を記録するための一連の演算 - `summary.scalar()`、`summary.histogram()`、`summary.image()`、`summary.audio()`、および `summary.text()`。これらはモデルコードからインラインで呼び出されます。
- 上記の個別の要約を収集して特別にフォーマットされたログファイル（TensorBoard が読み取って視覚化を生成するファイル）に書き込む書き込みロジック。

### TF 1.x の場合

The two halves had to be manually wired together - by fetching the summary op outputs via `Session.run()` and calling `FileWriter.add_summary(output, step)`.  The `v1.summary.merge_all()` op made this easier by using a graph collection to aggregate all summary op outputs, but this approach still worked poorly for eager execution and control flow, making it especially ill-suited for TF 2.x.

### TF 2.X の場合

上記の 2 つは密接に統合されており、個別の `tf.summary` 演算は実行時に直ちにデータを書き込むようになっています。モデルコードから API を使用する方法はあまり変わっていませんが、Eager execution との相性が改善されており、ほかのグラフモードとの互換性もそのままです。この 2 つの API を統合することで、`summary.FileWriter` は TensorFlow 実行コンテキストの一環となり、`tf.summary` 演算で直接アクセスできるため、ライターの構成が外見的な主な違いと言えます。

Example usage with eager execution, the default in TF 2.x:

In [ ]:
writer = tf.summary.create_file_writer("/tmp/mylogs/eager")

with writer.as_default():
  for step in range(100):
    # other model code would go here
    tf.summary.scalar("my_metric", 0.5, step=step)
    writer.flush()

In [ ]:
ls /tmp/mylogs/eager

次は、tf.function グラフ実行の使用例です。

In [ ]:
writer = tf.summary.create_file_writer("/tmp/mylogs/tf_function")

@tf.function
def my_func(step):
  with writer.as_default():
    # other model code would go here
    tf.summary.scalar("my_metric", 0.5, step=step)

for step in tf.range(100, dtype=tf.int64):
  my_func(step)
  writer.flush()

In [ ]:
ls /tmp/mylogs/tf_function

次は、レガシー TF 1.x グラフ実行の使用例です。


In [ ]:
g = tf.compat.v1.Graph()
with g.as_default():
  step = tf.Variable(0, dtype=tf.int64)
  step_update = step.assign_add(1)
  writer = tf.summary.create_file_writer("/tmp/mylogs/session")
  with writer.as_default():
    tf.summary.scalar("my_metric", 0.5, step=step)
  all_summary_ops = tf.compat.v1.summary.all_v2_summary_ops()
  writer_flush = writer.flush()


with tf.compat.v1.Session(graph=g) as sess:
  sess.run([writer.init(), step.initializer])

  for i in range(100):
    sess.run(all_summary_ops)
    sess.run(step_update)
    sess.run(writer_flush)  

In [ ]:
ls /tmp/mylogs/session

## コードを変換する

Converting existing `tf.summary` usage to the TF 2.x API cannot be reliably automated, so the [`tf_upgrade_v2` script](https://www.tensorflow.org/guide/upgrade) just rewrites it all to `tf.compat.v1.summary` and will not enable the TF 2.x behaviors automatically.

### Partial Migration

To make migration to TF 2.x easier for users of model code that still depends heavily on the TF 1.x summary API logging ops like `tf.compat.v1.summary.scalar()`, it is possible to migrate only the writer APIs first, allowing for individual TF 1.x summary ops inside your model code to be fully migrated at a later point.

To support this style of migration, <a href="https://www.tensorflow.org/api_docs/python/tf/compat/v1/summary"><code>tf.compat.v1.summary</code></a> will automatically forward to their TF 2.x equivalents under the following conditions:

 - The outermost context is eager mode
 - A default TF 2.x summary writer has been set
 - A non-empty value for step has been set for the writer (using <a href="https://www.tensorflow.org/api_docs/python/tf/summary/SummaryWriter#as_default"><code>tf.summary.SummaryWriter.as_default</code></a>, <a href="https://www.tensorflow.org/api_docs/python/tf/summary/experimental/set_step"><code>tf.summary.experimental.set_step</code></a>, or alternatively <a href="https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/create_global_step"><code>tf.compat.v1.train.create_global_step</code></a>)

Note that when TF 2.x summary implementation is invoked, the return value will be an empty bytestring tensor, to avoid duplicate summary writing. Additionally, the input argument forwarding is best-effort and not all arguments will be preserved (for instance `family` argument will be supported whereas `collections` will be removed).

Example to invoke <a href="https://www.tensorflow.org/api_docs/python/tf/summary/scalar"><code>tf.summary.scalar</code></a> behaviors in <a href="https://www.tensorflow.org/api_docs/python/tf/compat/v1/summary/scalar"><code>tf.compat.v1.summary.scalar</code></a>:

In [ ]:
# Enable eager execution.
tf.compat.v1.enable_v2_behavior()

# A default TF 2.x summary writer is available.
writer = tf.summary.create_file_writer("/tmp/mylogs/enable_v2_in_v1")
# A step is set for the writer.
with writer.as_default(step=0):
  # Below invokes `tf.summary.scalar`, and the return value is an empty bytestring.
  tf.compat.v1.summary.scalar('float', tf.constant(1.0), family="family")

### Full Migration

To fully migrate to TF 2.x, you'll need to adapt your code as follows:

1. サマリー演算を使用するには、`.as_default()` によるデフォルトのライターセットが存在する必要がある

    - つまり、演算を Eager で実行するか、グラフ構造で演算を使用する
    - デフォルトのライターがない場合、サマリー演算はサイレントの no-op になる
    - デフォルトのライターは（まだ）`@tf.function` 実行境界に伝搬しません。関数がトレースされた場合にのみ検出されるため、関数の本文で `writer.as_default()` を呼び出し、`@tf.function` が使用される限りライターオブジェクトが存在し続けられるようにすることが、ベストプラクティスと言えます。

2. 「step」値は `step` 引数で各演算に渡される必要がある

    - TensorBoard には、時系列としてデータをレンダリングするステップ値が必要です
    - TF 1.x のグローバルステップは削除されており、各演算は読み取るための希望する step 変数を知っておくために、明示的に渡す必要があります
    - ボイラープレートを減らすために、デフォルトステップを登録するための実験的サポートは`tf.summary.experimental.set_step()` として提供されていますが、これは暫定機能であり、予告なく変更される場合があります

3. 個々のサマリー演算の関数シグネチャが変更されている

    - 戻り値はブール型になっています（要約が実際に書き込まれたかどうかを示す）
    - 2 番目のパラメータ名（使用される場合）が`tensor` から `data` に代わっています
    - `collections` パラメータが削除されています。collections は TF 1.x のみのパラメータです
    - `family` パラメータが削除されています。`tf.name_scope()` を使用してください

4. ［レガシーグラフモードのみ / セッション実行ユーザー］

    - 最初に `v1.Session.run(writer.init())` でライターを初期化します

    - Use `v1.summary.all_v2_summary_ops()` to get all TF 2.x summary ops for the current graph, e.g. to execute them via `Session.run()`

    - `v1.Session.run(writer.flush())` でライターをフラッシュし、`close()` でも同様にフラッシュします

If your TF 1.x code was instead using `tf.contrib.summary` API, it's much more similar to the TF 2.x API, so `tf_upgrade_v2` script will automate most of the migration steps (and emit warnings or errors for any usage that cannot be fully migrated).  For the most part it just rewrites the API calls to `tf.compat.v2.summary`; if you only need compatibility with TF 2.x you can drop the `compat.v2` and just reference it as `tf.summary`.

## その他のヒント

上記の重要な分野に加え、一部の補助的な側面も変更されています。

- 条件付き記録（「100 ステップごとにログ」など）が新しくなりました

    - 演算と関連するコードを制御するには、通常の if ステートメント（Eager モードと[自動グラフ作成経由の `@tf.function`](https://www.tensorflow.org/alpha/guide/autograph) で機能）か、`tf.cond` でラップします
    - 要約のみを制御するには、新しい `tf.summary.record_if()` コンテキストマネージャを使用して、選択したブール条件を渡します
    - TF 1.x パターンを置き換えます
        ```
        if condition:
          writer.add_summary()
        ```


- `tf.compat.v1.Graph` を直接書き込めない代わりにトレース関数を使用します

    - Graph execution in TF 2.x uses `@tf.function` instead of the explicit Graph
    - In TF 2.x, use the new tracing-style APIs `tf.summary.trace_on()` and `tf.summary.trace_export()` to record executed function graphs


- `tf.summary.FileWriterCache` の使用により、logdir ごとのグローバルライターのキャッシュが不要になります

    - ユーザーは、ライターオブジェクトの独自のキャッシュ/共有を実装するか、個別のライターを使用する必要があります（TensorBoard の後者のサポートは[開発中](https://github.com/tensorflow/tensorboard/issues/1063)です）


- イベントファイルのバイナリ表現が変更されました

    - TensorBoard 1.x は、新しい形式をサポート済みです。この違いは、要約データをイベントファイルから手動で解析しているユーザーのみに影響します）
    - 要約データはテンソルバイトとして保存されるようになりました。`tf.make_ndarray(event.summary.value[0].tensor)` を使って numpy に変換できます。